## Sample-and-Aggregate algorithm using our DP Mean and Gaussian Mechanism


In [1]:
import numpy as np
import pandas as pd

# established in previous lectures
#from mock_dp_library import *

In [3]:
# introduced in wk3
# functions used regularly in examples

import numpy as np


def laplace(shift=0., scale=1., size=None):
    """Sample from the laplace distribution."""
    p = np.random.uniform(low=-0.5, high=0.5, size=size)
    draws = shift - scale * np.sign(p) * np.log(1 - 2 * abs(p))
    return draws

    # the easy way
    # return np.random.laplace(loc=shift, scale=scale, size=size)


def gaussian(shift=0., scale=1., size=None):
    """Sample from the Gaussian distribution."""
    draws = np.random.normal(loc=shift, scale=scale, size=size)
    return draws


def clamp(x, bounds):
    """Replace any x_i less than lower with lower,
           and any x_i greater than upper with upper."""
    return np.clip(x, *bounds)


def bounded_mean(x, bounds):
    x_clamped = clamp(x, bounds)
    return np.mean(x_clamped)


def release_dp_mean(x, bounds, epsilon, delta=1e-6, mechanism="laplace"):
    """Release a DP mean.
    Assumes that the dataset size n is public information.
    """
    sensitive_mean = bounded_mean(x, bounds)

    n = len(x)
    lower, upper = bounds
    # Sensitivity in terms of an absolute distance metric
    # Both the laplace and gaussian mechanisms can noise queries
    #    with sensitivities expressed in absolute distances
    sensitivity = (upper - lower) / n

    if mechanism == "laplace":
        scale = sensitivity / epsilon
        dp_mean = sensitive_mean + laplace(scale=scale)
    elif mechanism == "gaussian":
        scale = (sensitivity / epsilon) * np.sqrt(2*np.log(2/delta))
        dp_mean = sensitive_mean + gaussian(scale=scale)
    else:
        raise ValueError(f"unrecognized mechanism: {mechanism}")

    return dp_mean


def bootstrap(x, n):
    """Sample n values with replacement from n."""
    index = np.random.randint(low=0., high=len(x), size=n)
    return x[index]


def release_dp_histogram(x, epsilon, categories):
    """Release an `epsilon`-DP estimate of the counts of each of the `categories`"""
    sensitivity = 2
    scale = sensitivity / epsilon

    # create a {category: count} hashmap
    counts = dict(zip(*np.unique(x, return_counts=True)))
    # look up the count of each category, or zero if not exists
    sensitive_histogram = np.array([counts.get(cat, 0) for cat in categories])

    dp_histogram = sensitive_histogram + laplace(scale=scale, size=sensitive_histogram.shape)
    return dp_histogram

Read in the data.  We're going to use the PUMS dataset we are familiar with, and focus on the education variable, a 16 point scale.

In [4]:
import pandas as pd
data = pd.read_csv(
    "https://raw.githubusercontent.com/opendp/cs208/main/spring2022/data/FultonPUMS5full.csv")

# define public information
n = len(data)            # in this case, dataset length is considered public, and is not protected
educ_bounds = (1., 16.)  # easily guessable without looking at the data

educ = data['educ'].values.astype(float)
print(release_dp_mean(educ, bounds=educ_bounds, epsilon=1.))

10.608234813359998


In [5]:
def sample_aggregate(data: pd.DataFrame, function, partition_count: int, bounds, epsilon, delta):

    ## SAMPLE
    # shuffle without replacement
    data = data.sample(frac=1, replace=False)
    # split data into `partition_count` datasets
    partitions = np.array_split(data, partition_count)

    ## EVALUATE
    results = []
    for partition in partitions:
        results.append(function(partition))

    ## AGGREGATE
    private_release = release_dp_mean(
        results, bounds=bounds, epsilon=epsilon, delta=delta, mechanism="gaussian")

    return(private_release)

In [6]:
def correlation(data):
    return np.corrcoef(data['educ'], data['income'])[0, 1]


dp_correlation = sample_aggregate(
    data, correlation, partition_count=200, bounds=[0,1], epsilon=1, delta=1e-6)

print("True correlation:", correlation(data))
print("DP   correlation:", dp_correlation)

True correlation: 0.35472882626591723
DP   correlation: 0.34310199984026224


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
